In [3]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb rank_bm25 faiss-cpu langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


## Hybrid Search
### BM25 Retriever - Sparse Retreiver

In [2]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

<ipython-input-2-938344147>:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [19]:
doc_list = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
    "I like computers by Apple",
    "I love fruit juice",
    "Samsung is a chinese company which makes smart phones",
    "China mobiles are so cheap",
    "India import Chinese company mobiles"
]

In [20]:
# initialize the bm25 retriever and faiss retreiver
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

In [21]:
bm25_retriever.get_relevant_documents("Apple")

[Document(metadata={}, page_content='I like computers by Apple'),
 Document(metadata={}, page_content='India import Chinese company mobiles')]

In [22]:
bm25_retriever.get_relevant_documents("a green fruit")

[Document(metadata={}, page_content='I love fruit juice'),
 Document(metadata={}, page_content='Samsung is a chinese company which makes smart phones')]

## Embeddings - Dense retrievers FAISS

In [23]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
import torch
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

# Detect device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize embeddings
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs=encode_kwargs
)

In [24]:
faiss_vectorstore = FAISS.from_texts(doc_list, bge_embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

In [25]:
faiss_retriever.get_relevant_documents("i like green fruits")

[Document(id='d04cd9d8-eba2-4530-a810-99216c09e493', metadata={}, page_content='I like apples'),
 Document(id='db093eac-3f57-4a48-9fc2-e9c1f572c137', metadata={}, page_content='I love fruit juice')]

## Ensemble Retriever

In [26]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5]
)


In [27]:
docs = ensemble_retriever.get_relevant_documents("i love chinese mobiles")
docs

[Document(metadata={}, page_content='I love fruit juice'),
 Document(id='c8b13141-6c9d-46af-8e95-d1c635afd936', metadata={}, page_content='China mobiles are so cheap'),
 Document(metadata={}, page_content='Samsung is a chinese company which makes smart phones'),
 Document(id='8e6ca393-cfc0-40ce-ab34-25803c97cca0', metadata={}, page_content='India import Chinese company mobiles')]

In [28]:
docs = ensemble_retriever.get_relevant_documents("Apple Phones")
docs

[Document(metadata={}, page_content='I like computers by Apple'),
 Document(metadata={}, page_content='India import Chinese company mobiles'),
 Document(id='d04cd9d8-eba2-4530-a810-99216c09e493', metadata={}, page_content='I like apples')]